In [1]:
import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

cache_seed = 41 # default seed is 41


my_models = [
    {
        'model': os.environ['GPT4-1106-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT432-0613-32k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT4-0125PREVIEW-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {
            "gpt4-0125preview-128k"
        }
    }
)

config_list

[{'model': 'gpt4-0125preview-128k',
  'api_key': '5948e5b2b4a146cba9940adb3308d731',
  'base_url': 'https://mmopenaiscus.openai.azure.com/',
  'api_type': 'azure',
  'api_version': '2024-02-15-preview'}]

In [2]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    }
)

user_proxy

In [3]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

assistant

In [4]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description


chat_res = user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

First, let's find out today's date using Python. After that, we will proceed to compare the year-to-date (YTD) gain for META (Facebook's parent company, Meta Platforms, Inc.) and TESLA (Tesla, Inc.).

```python
from datetime import datetime

# Get today's date
today = datetime.now()

print(f"Today's date is: {today.strftime('%Y-%m-%d')}")
```

Please execute this code to get today's date. After that, we will move on to the next step of comparing the YTD gains for META and TESLA.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Today's date is: 2024-03-28


------------------------------------------------------

In [5]:
# autogen.Completion.clear_cache(seed=cache_seed)

In [6]:
i=1
for cr in chat_res.chat_history:
    print(i, cr["content"].rstrip()[-9:])
    i=i+1

1 nd TESLA.
2 nd TESLA.
3 024-03-28
4 e output.
5 : -28.09%
6 TERMINATE
